In [1]:
import pandas as pd
import numpy as np
import ordpy
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("iot_23.csv", sep=";")
df

,Unnamed: 0.1,Unnamed: 0,frame_time,total_pacotes,total_pacotes_icmp,total_pacotes_udp,total_pacotes_tcp,maior_pacote,menor_pacote,soma_pacotes,total_ips_origem,total_ips_destino,porta_origem_mais_frequente,porta_destino_mais_frequente,has_bot,has_bot_origem_destino
0,0,0,"0 Sep 6, 2018 06:43:20.739949000 -03\nName...",47707,8,0,47574,238,60,2872172,28,23690,0,0,0,0
1,1,1,"0 Sep 6, 2018 06:44:20.708723000 -03\nName...",75612,12,0,75560,90,60,4537976,27,37770,0,0,0,0
2,2,2,"0 Sep 6, 2018 06:45:20.280264000 -03\nName...",76304,20,0,76240,90,60,4579840,36,38095,0,0,0,0
3,3,3,"0 Sep 6, 2018 06:46:20.000233000 -03\nName...",75926,16,0,75874,110,60,4556972,33,37916,0,0,0,0
4,4,4,"0 Sep 6, 2018 06:47:20.296901000 -03\nName...",75620,18,0,75558,90,60,4538568,32,37764,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,1435,28,"0 Sep 7, 2018 06:38:20.237516000 -03\nName...",75962,14,0,75908,182,60,4559568,20,37923,0,0,1,1
1436,1436,29,"0 Sep 7, 2018 06:39:20.133381000 -03\nName...",75976,10,0,75914,90,60,4560370,18,37922,0,0,1,1
1437,1437,30,"0 Sep 7, 2018 06:40:20.124184000 -03\nName...",76266,2,0,76208,90,60,4577586,14,38084,0,0,1,1
1438,1438,31,"0 Sep 7, 2018 06:41:20.251381000 -03\nName...",75962,12,0,75888,90,60,4559672,15,37919,0,0,1,1


# Loop para extrair as métricas sem variar o taux

In [3]:
matrix = []
label = []
cols = ['total_ips_origem','total_ips_destino', 'maior_pacote', 
        'total_pacotes_icmp','total_pacotes_tcp'
       ]
taux = 1
dx = 3

init = 0
end = 144 

for i in range (1, end):
    matrix.append([[init, i]])

for i in range(0,1296): 
    vec = []
    vec.append([init, end])
    
    for col in cols:
        tmp = ordpy.complexity_entropy(df[col][init:end], dx=dx, taux=taux)
        vec.append(tmp[0])
        vec.append(tmp[1])
    
        tmp = ordpy.fisher_shannon(df[col][init:end], dx=dx, taux=taux)
        vec.append(tmp[1])
    
        edges = ordpy.ordinal_network(df[col][init:end], dx=dx, taux=taux)

        tmp = ordpy.complexity_entropy(edges[2], dx=dx, taux=taux)
        vec.append(tmp[0])
        vec.append(tmp[1])
    
        tmp = ordpy.fisher_shannon(edges[2], dx=dx, taux=taux)
        vec.append(tmp[1])
    
        tmp = len(edges[1])
        vec.append(tmp)
    
        j=0
        indices = []
        for array in edges[1]:
            if(array[0] == array[1]):
                indices.append(j)
            j += 1
    
        prob = 0
        for index in indices:
            tmp = edges[2][index]
            prob += tmp
        vec.append(prob)

    matrix.append(vec)

    bot = 0
    for line in df.iloc[init:end].iterrows():
        if(line[1][4] == 1):
            bot = 1
    label.append(bot)
        
    init = init + 1
    end = end + 1

In [4]:
ft = pd.DataFrame(matrix, columns = ['time',
                                     'IPO_entropy', 'IPO_complexity' , 'IPO_fisher_shanon', 'IPO_edge_entropy', 'IPO_edge_complexity', 'IPO_edge_fisher_shannon', 'IPO_number_nodes', 'IPO_self_transition',
                                     'IPD_entropy', 'IPD_complexity' , 'IPD_fisher_shanon', 'IPD_edge_entropy', 'IPD_edge_complexity', 'IPD_edge_fisher_shannon', 'IPD_number_nodes', 'IPD_self_transition',
                                     'MP_entropy', 'MP_complexity' , 'MP_fisher_shanon', 'MP_edge_entropy', 'MP_edge_complexity', 'MP_edge_fisher_shannon', 'MP_number_nodes', 'MP_self_transition',
                                     'TP_ICMP_entropy', 'TP_ICMP_complexity' , 'TP_ICMP_fisher_shanon', 'TP_ICMP_edge_entropy', 'TP_ICMP_edge_complexity', 'TP_ICMP_edge_fisher_shannon', 'TP_ICMP_number_nodes', 'TP_ICMP_self_transition',
                                     'TP_TCP_entropy', 'TP_TCP_complexity' , 'TP_TCP_fisher_shanon', 'TP_TCP_edge_entropy', 'TP_TCP_edge_complexity', 'TP_TCP_edge_fisher_shannon', 'TP_TCP_number_nodes', 'TP_TCP_self_transition'])
ft
ft.to_csv('po_iot_23.csv',sep=';')
ft[0:50]

,time,IPO_entropy,IPO_complexity,IPO_fisher_shanon,IPO_edge_entropy,IPO_edge_complexity,IPO_edge_fisher_shannon,IPO_number_nodes,IPO_self_transition,IPD_entropy,...,TP_ICMP_number_nodes,TP_ICMP_self_transition,TP_TCP_entropy,TP_TCP_complexity,TP_TCP_fisher_shanon,TP_TCP_edge_entropy,TP_TCP_edge_complexity,TP_TCP_edge_fisher_shannon,TP_TCP_number_nodes,TP_TCP_self_transition
0,"[0, 1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[0, 2]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[0, 3]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[0, 4]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[0, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"[0, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"[0, 7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"[0, 8]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"[0, 9]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"[0, 10]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
import numpy as np
import time
import timeit

po = pd.read_csv("po_iot_23.csv", sep=";")
early_warning = pd.read_csv("iot_23.csv", sep=";")

df = po[['IPO_entropy', 'IPO_complexity',
       'IPO_fisher_shanon', 'IPO_edge_entropy', 'IPO_edge_complexity',
       'IPO_edge_fisher_shannon', 'IPO_number_nodes', 'IPO_self_transition',
       'IPD_entropy', 'IPD_complexity', 'IPD_fisher_shanon',
       'IPD_edge_entropy', 'IPD_edge_complexity', 'IPD_edge_fisher_shannon',
       'IPD_number_nodes', 'IPD_self_transition', 'MP_entropy',
       'MP_complexity', 'MP_fisher_shanon', 'MP_edge_entropy',
       'MP_edge_complexity', 'MP_edge_fisher_shannon', 'MP_number_nodes',
       'MP_self_transition',
       'TP_ICMP_entropy', 'TP_ICMP_complexity' , 'TP_ICMP_fisher_shanon', 'TP_ICMP_edge_entropy', 'TP_ICMP_edge_complexity', 'TP_ICMP_edge_fisher_shannon', 'TP_ICMP_number_nodes', 'TP_ICMP_self_transition',
       'TP_TCP_entropy', 'TP_TCP_complexity' , 'TP_TCP_fisher_shanon', 'TP_TCP_edge_entropy', 'TP_TCP_edge_complexity', 'TP_TCP_edge_fisher_shannon', 'TP_TCP_number_nodes', 'TP_TCP_self_transition']]
label = early_warning['has_bot']

init = 144
train = 221
end = 351

x_train = df[init:train]
x_test = df[train:end]
y_real = label[train:end]

clf = OneClassSVM(gamma='auto', kernel='poly', nu=0.1).fit(x_train)
fim = timeit.default_timer()

y_test = clf.predict(x_test)
y_test_final = [0 if i == 1 else 1 for i in y_test]
acc = accuracy_score(y_real, y_test_final)
print(acc)
print(confusion_matrix(y_real, y_test_final))
print(classification_report(y_real, y_test_final, digits = 4))

0.7230769230769231
[[79  1]
 [35 15]]
              precision    recall  f1-score   support

           0     0.6930    0.9875    0.8144        80
           1     0.9375    0.3000    0.4545        50

    accuracy                         0.7231       130
   macro avg     0.8152    0.6438    0.6345       130
weighted avg     0.7870    0.7231    0.6760       130

